In [5]:
import json
import requests
import pandas as pd
from geopy.distance import geodesic


In [8]:
def find_postal(lst, filename):
    
    for index,add in enumerate(lst):
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+add
        #print(index,url)
        response = requests.get(url)
        data = json.loads(response.text) 
        temp_df = pd.DataFrame.from_dict(data["results"])
        temp_df["address"] = add
    
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')


In [18]:
non_landed = pd.read_csv('Non-landed.csv', encoding='ISO-8859-1')
non_landed.head()
unique_address = non_landed['Project Name'].unique()

find_postal(unique_address, "non_landed_geoloc_1")

In [21]:
non_landed_postal = pd.read_csv('non_landed_geoloc_1.csv')
non_landed_postal = non_landed_postal.drop_duplicates(subset=['address'], keep='first')
df_non_landed_postal= pd.merge(non_landed, non_landed_postal, how='left', left_on=['Project Name'], right_on=['address']).dropna()
df_non_landed_postal.to_csv('non_landed_withpostal.csv')

In [22]:
def find_nearest(house, amenity):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    add_lat_lon_start, add_lat_lon_end --> both is a data frame with a specific format:
        1st column: any string column ie addresses, mrt stations
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])

            distance = geodesic(flat_loc,amenity_loc)

            if distance < flat_amenity[2]:
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [23]:
# hdb resale dataset with location
geo = pd.read_csv("non_landed_withpostal.csv")
geo1 = geo[['address', 'LATITUDE', 'LONGITUDE']]


data = [['cityhall', 1.29317576, 103.8525073]] 
sgcentre = pd.DataFrame(data, columns = ['address', 'LATITUDE', 'LONGITUDE'])
sgcentre
map_sgcentre = find_nearest(geo1,sgcentre)
map_sgcentre1 = pd.DataFrame(map_sgcentre)
map_sgcentre2 = map_sgcentre1.transpose()
map_sgcentre2.to_csv('city_NEW.csv',index = False)

supermarket = pd.read_csv("supermarket_geoloc.csv")
supermarket1 = supermarket[['Supermarket','LATITUDE', 'LONGITUDE']]
map_supermarket = find_nearest(geo1,supermarket1)
map_supermarket1 = pd.DataFrame(map_supermarket)
map_supermarket2 = map_supermarket1.transpose()
map_supermarket2.to_csv('supermarket_NEW.csv',index = False)

hawker_ = pd.read_csv("hawker_geoloc.csv")
hawker_1 = hawker_[['HAWKER','LATITUDE', 'LONGITUDE']]
map_hawker_ = find_nearest(geo1,hawker_1)
map_hawker_1 = pd.DataFrame(map_hawker_)
map_hawker_2 = map_hawker_1.transpose()
map_hawker_2.to_csv('hawker_NEW.csv',index = False)

npc = pd.read_csv("npc_geoloc.csv.csv")
npc1 = npc[['NPC','LATITUDE', 'LONGITUDE']]
map_npc = find_nearest(geo1,npc1)
map_npc1 = pd.DataFrame(map_npc)
map_npc2 = map_npc1.transpose()
map_npc2.to_csv('npc_NEW.csv',index = False)

ps = pd.read_csv("primaryschoolsg.csv", encoding='mac_roman')
ps1 = ps[['Name', 'Latitude','Longitude']]
map_ps = find_nearest(geo1,ps1)
map_ps1 = pd.DataFrame(map_ps)
map_ps2 = map_ps1.transpose()
map_ps2.to_csv('ps_NEW.csv',index = False)

ss = pd.read_csv("secsch_cleaned_1.csv", encoding='mac_roman')
ss1 = ss[['SCHNAME', 'latitude','longitude']]
map_ss = find_nearest(geo1,ss1)
map_ss1 = pd.DataFrame(map_ss)
map_ss2 = map_ss1.transpose()
map_ss2.to_csv('SS_NEW.csv',index = False)

cc = pd.read_csv("cc_geoloc.csv")
cc1 = cc[['CC', 'LATITUDE', 'LONGITUDE']]
map_cc = find_nearest(geo1,cc1)
map_cc1 = pd.DataFrame(map_cc)
map_cc2 = map_cc1.transpose()
map_cc2.to_csv('CC_NEW.csv',index = False)

# For Mrt - # find nearest train station to the flat
mrt = pd.read_csv("mrtsg.csv")
mrt1 = mrt[['STN_NAME', 'Latitude', 'Longitude']]
map_mrt = find_nearest(geo1,mrt1)
map_mrt1 = pd.DataFrame(map_mrt)
map_mrt2 = map_mrt1.transpose()
map_mrt2.to_csv('MRT_NEW.csv',index = False)
